# Phase 4 – Bias Validation & Reclassification Quality Check (BRCA2 South Asian Bias Project)

This notebook validates your **ACMG frequency-based reclassification** from Phase 3 and
examines **database bias** across populations.

**Inputs (from previous phases)**:
- `data/processed/brca2_merged.csv`  – full merged BRCA2 dataset
- `data/processed/brca2_reclass_candidates.csv` – VUS/Pathogenic candidates with BA1/BS1 flags
- `results/tables/table1_acmg_reclassifications.csv` – clean reclassification table

**Outputs (this notebook)**:
- `results/tables/table2_population_presence_by_significance.csv`
- `results/tables/table3_reviewstatus_by_significance.csv`
- `results/tables/table4_sas_eur_ratio_summary.csv`
- `results/tables/table4_reclassification_summary.csv`
- `results/figures/figure3_population_af_boxplots.png`
- `results/figures/figure4_reviewstatus_sas_af_boxplot.png`
- `results/figures/figure5_sas_eur_ratio_histogram.png`


## 0. Environment setup

Run this cell once per session.


In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("✅ Imports done. pandas version:", pd.__version__)


✅ Imports done. pandas version: 2.2.2


In [2]:
# If you are running in Google Colab, mount your Drive.
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")
    print("✅ Google Drive mounted.")
except ModuleNotFoundError:
    print("ℹ️ Not running in Colab; skipping Drive mount.")


Mounted at /content/drive
✅ Google Drive mounted.


In [3]:
PROJECT_ROOT = "/content/drive/MyDrive/BRCA2-database-bias"

DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
RESULTS_TABLES = os.path.join(PROJECT_ROOT, "results", "tables")
RESULTS_FIGURES = os.path.join(PROJECT_ROOT, "results", "figures")

os.makedirs(RESULTS_TABLES, exist_ok=True)
os.makedirs(RESULTS_FIGURES, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_PROCESSED:", DATA_PROCESSED)
print("RESULTS_TABLES:", RESULTS_TABLES)
print("RESULTS_FIGURES:", RESULTS_FIGURES)


PROJECT_ROOT: /content/drive/MyDrive/BRCA2-database-bias
DATA_PROCESSED: /content/drive/MyDrive/BRCA2-database-bias/data/processed
RESULTS_TABLES: /content/drive/MyDrive/BRCA2-database-bias/results/tables
RESULTS_FIGURES: /content/drive/MyDrive/BRCA2-database-bias/results/figures


## 1. Load Phase 1 & 3 outputs

We now load:

- The **merged BRCA2 dataset** from Phase 1
- The **reclassification candidates** from Phase 3
- The **clean Table 1** generated in Phase 3


In [4]:
merged_path = os.path.join(DATA_PROCESSED, "brca2_merged.csv")
cands_path = os.path.join(DATA_PROCESSED, "brca2_reclass_candidates.csv")
table1_path = os.path.join(RESULTS_TABLES, "table1_acmg_reclassifications.csv")

df = pd.read_csv(merged_path)
cands = pd.read_csv(cands_path)
table1 = pd.read_csv(table1_path)

print("✅ Loaded datasets:")
print("  merged:", df.shape)
print("  candidates:", cands.shape)
print("  Table 1:", table1.shape)

df.head()


✅ Loaded datasets:
  merged: (20614, 21)
  candidates: (1, 29)
  Table 1: (1, 10)


,Chromosome,Start,ReferenceAllele,AlternateAllele,ClinicalSignificance,ReviewStatus,variant_key,variant_id,chrom,pos,...,alt,consequence,sas_af,eur_af,afr_af,eas_af,amr_af,log10_sas_af,log10_eur_af,sas_eur_ratio
0,13,32315212,G,A,"('Benign',)","('criteria_provided', '_single_submitter')",13-32315212-G-A,13-32315212-G-A,13.0,32315212.0,...,A,intron_variant,0.000000,0.000015,0.002792,0.000000,0.000262,-12.000000,-4.832522,6.800200e-08
1,13,32315226,G,A,"('Benign',)","('reviewed_by_expert_panel',)",13-32315226-G-A,13-32315226-G-A,13.0,32315226.0,...,A,intron_variant,0.250000,0.177842,0.500000,0.149612,0.201544,-0.602060,-0.749965,1.405741e+00
2,13,32315300,G,A,"('Benign',)","('criteria_provided', '_single_submitter')",13-32315300-G-A,13-32315300-G-A,13.0,32315300.0,...,A,intron_variant,0.000415,0.005453,0.000650,0.000193,0.001961,-3.382377,-2.263352,7.602821e-02
3,13,32315355,A,G,"('Uncertain_significance',)","('criteria_provided', '_single_submitter')",13-32315355-A-G,13-32315355-A-G,13.0,32315355.0,...,G,5_prime_UTR_variant,0.000000,0.000000,0.000000,0.000965,0.000000,-12.000000,-12.000000,1.000000e+00
4,13,32315355,ATGCCTGACAAGGAATTTCCTTTCGCCACACTGAGAAATACCCGCA...,A,"('Pathogenic',)","('no_assertion_criteria_provided',)",13-32315355-ATGCCTGACAAGGAATTTCCTTTCGCCACACTGA...,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,-12.000000,-12.000000,1.000000e+00


In [5]:
# Ensure key columns exist, create if missing so we don't crash later
pop_cols = ["sas_af", "eur_af", "afr_af", "eas_af", "amr_af"]
meta_cols = ["ClinicalSignificance", "ReviewStatus", "sas_eur_ratio", "variant_key"]

for col in pop_cols + meta_cols:
    if col not in df.columns:
        df[col] = np.nan

df[["ClinicalSignificance"] + pop_cols].head()


,ClinicalSignificance,sas_af,eur_af,afr_af,eas_af,amr_af
0,"('Benign',)",0.000000,0.000015,0.002792,0.000000,0.000262
1,"('Benign',)",0.250000,0.177842,0.500000,0.149612,0.201544
2,"('Benign',)",0.000415,0.005453,0.000650,0.000193,0.001961
3,"('Uncertain_significance',)",0.000000,0.000000,0.000000,0.000965,0.000000
4,"('Pathogenic',)",0.000000,0.000000,0.000000,0.000000,0.000000


## 2. Population presence by clinical significance

Question: **Are certain ClinVar categories (e.g., VUS) more likely to be missing South Asian data?**

We will:

1. Create "presence" flags (AF > 0) for each population.
2. Summarize by `ClinicalSignificance`.
3. Save as Table 2.


In [6]:
# Presence flags: AF > 0
for col in pop_cols:
    presence_col = col.replace("_af", "_present")
    df[presence_col] = df[col].fillna(0) > 0

presence_cols = [c for c in df.columns if c.endswith("_present")]

table2 = (
    df.groupby("ClinicalSignificance")[presence_cols]
    .mean()
    .reset_index()
)

# Convert proportions to percentages for readability
for c in presence_cols:
    table2[c] = (table2[c] * 100).round(1)

table2_path = os.path.join(RESULTS_TABLES, "table2_population_presence_by_significance.csv")
table2.to_csv(table2_path, index=False)
print("✅ Saved Table 2 to:", table2_path)

table2.head()


✅ Saved Table 2 to: /content/drive/MyDrive/BRCA2-database-bias/results/tables/table2_population_presence_by_significance.csv


,ClinicalSignificance,sas_present,eur_present,afr_present,eas_present,amr_present
0,"('Benign',)",59.1,85.9,80.2,46.1,79.4
1,"('Benign/Likely_benign',)",34.9,77.1,46.8,22.0,48.6
2,"('Conflicting_classifications_of_pathogenicity',)",8.4,31.3,9.6,5.5,7.0
3,"('Likely_benign',)",6.7,23.0,6.8,3.5,4.7
4,"('Likely_pathogenic',)",1.5,4.3,0.9,0.9,0.9


## 3. Population AF distributions (boxplots)

We visualize the overall AF distribution by population to see if **South Asian AF values** are systematically lower
or sparser than other groups.

Output:

- `results/figures/figure3_population_af_boxplots.png`


In [7]:
# Prepare data for boxplot: filter to AF > 0 to avoid cluttering with zeros
af_data = []
labels = []
for col in pop_cols:
    series = df[col].dropna()
    series = series[series > 0]
    if len(series) > 0:
        af_data.append(series)
        labels.append(col.replace("_af", "").upper())

if af_data:
    plt.figure(figsize=(8, 5))
    plt.boxplot(af_data, showfliers=False)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.yscale("log")
    plt.ylabel("Allele frequency (log scale)")
    plt.title("Figure 3: Population allele frequency distributions (BRCA2 variants)")

    fig3_path = os.path.join(RESULTS_FIGURES, "figure3_population_af_boxplots.png")
    plt.savefig(fig3_path, dpi=200, bbox_inches="tight")
    plt.close()

    print("✅ Saved Figure 3 to:", fig3_path)
else:
    print("⚠️ No non-zero AF values found; Figure 3 was not created.")


✅ Saved Figure 3 to: /content/drive/MyDrive/BRCA2-database-bias/results/figures/figure3_population_af_boxplots.png


## 4. Review status vs SAS AF (bias in evidence quality)

We examine whether **high SAS AF variants** are stuck with **low review status** (e.g., single submitter).

Steps:

1. For each `ReviewStatus`, compute the distribution of `sas_af`.
2. Plot a boxplot of `sas_af` by review status.
3. Save as Table 3 + Figure 4.


In [8]:
# Ensure ReviewStatus exists
if "ReviewStatus" not in df.columns:
    df["ReviewStatus"] = "Unknown"

# Table 3: summary of review status by significance
table3 = (
    df.groupby(["ClinicalSignificance", "ReviewStatus"])["sas_af"]
    .agg(count="size", mean_sas_af="mean")
    .reset_index()
)

table3_path = os.path.join(RESULTS_TABLES, "table3_reviewstatus_by_significance.csv")
table3.to_csv(table3_path, index=False)
print("✅ Saved Table 3 to:", table3_path)

table3.head()


✅ Saved Table 3 to: /content/drive/MyDrive/BRCA2-database-bias/results/tables/table3_reviewstatus_by_significance.csv


,ClinicalSignificance,ReviewStatus,count,mean_sas_af
0,"('Benign',)","('criteria_provided', '_multiple_submitters', ...",5,0.200002
1,"('Benign',)","('criteria_provided', '_single_submitter')",71,0.004580
2,"('Benign',)","('no_assertion_criteria_provided',)",6,0.000002
3,"('Benign',)","('reviewed_by_expert_panel',)",696,0.091987
4,"('Benign/Likely_benign',)","('criteria_provided', '_multiple_submitters', ...",108,0.000501


In [9]:
# Figure 4: SAS AF by ReviewStatus (boxplot, log scale)
review_data = []
review_labels = []

for status, group in df.groupby("ReviewStatus"):
    vals = group["sas_af"].dropna()
    vals = vals[vals > 0]
    if len(vals) == 0:
        continue
    review_data.append(vals)
    review_labels.append(status)

if review_data:
    plt.figure(figsize=(9, 5))
    plt.boxplot(review_data, showfliers=False)
    plt.xticks(range(1, len(review_labels) + 1), review_labels, rotation=45, ha="right")
    plt.yscale("log")
    plt.ylabel("South Asian AF (sas_af, log scale)")
    plt.title("Figure 4: SAS AF distribution by ClinVar review status")

    fig4_path = os.path.join(RESULTS_FIGURES, "figure4_reviewstatus_sas_af_boxplot.png")
    plt.savefig(fig4_path, dpi=200, bbox_inches="tight")
    plt.close()

    print("✅ Saved Figure 4 to:", fig4_path)
else:
    print("⚠️ No SAS AF values available for review status plot; Figure 4 was not created.")


✅ Saved Figure 4 to: /content/drive/MyDrive/BRCA2-database-bias/results/figures/figure4_reviewstatus_sas_af_boxplot.png


## 5. SAS/EUR ratio (cross-population skew)

We examine how often SAS AF is much higher than EUR AF.

Steps:

1. Ensure `sas_eur_ratio` exists (compute if missing).
2. Plot histogram of `sas_eur_ratio` (log scale).
3. Save as Figure 5 and a small summary table.

This directly quantifies **database bias against South Asians** when EUR is used as the "default".


In [10]:
# Ensure sas_eur_ratio exists
if "sas_eur_ratio" not in df.columns:
    df["sas_eur_ratio"] = (df["sas_af"].fillna(0) + 1e-12) / (df["eur_af"].fillna(0) + 1e-12)

ratio = df["sas_eur_ratio"].replace([np.inf, -np.inf], np.nan).dropna()

if len(ratio) > 0:
    plt.figure(figsize=(7, 4))
    plt.hist(np.log10(ratio), bins=40)
    plt.xlabel("log10(SAS AF / EUR AF)")
    plt.ylabel("Count")
    plt.title("Figure 5: Distribution of SAS/EUR allele frequency ratios (BRCA2 variants)")

    fig5_path = os.path.join(RESULTS_FIGURES, "figure5_sas_eur_ratio_histogram.png")
    plt.savefig(fig5_path, dpi=200, bbox_inches="tight")
    plt.close()

    print("✅ Saved Figure 5 to:", fig5_path)
else:
    print("⚠️ No valid SAS/EUR ratios found; Figure 5 was not created.")

# Simple summary table of ratio categories
def ratio_bucket(x):
    if x < 0.2:
        return "<0.2x (lower in SAS)"
    elif x < 1:
        return "0.2–1x"
    elif x < 5:
        return "1–5x"
    elif x < 10:
        return "5–10x"
    else:
        return ">=10x (higher in SAS)"

df["sas_eur_ratio_bucket"] = df["sas_eur_ratio"].apply(ratio_bucket)

ratio_summary = (
    df["sas_eur_ratio_bucket"]
    .value_counts()
    .rename_axis("bucket")
    .reset_index(name="count")
)

ratio_summary_path = os.path.join(RESULTS_TABLES, "table4_sas_eur_ratio_summary.csv")
ratio_summary.to_csv(ratio_summary_path, index=False)
print("✅ Saved SAS/EUR ratio summary table to:", ratio_summary_path)

ratio_summary


✅ Saved Figure 5 to: /content/drive/MyDrive/BRCA2-database-bias/results/figures/figure5_sas_eur_ratio_histogram.png
✅ Saved SAS/EUR ratio summary table to: /content/drive/MyDrive/BRCA2-database-bias/results/tables/table4_sas_eur_ratio_summary.csv


,bucket,count
0,1–5x,15610
1,<0.2x (lower in SAS),3802
2,>=10x (higher in SAS),876
3,0.2–1x,246
4,5–10x,80


## 6. Reclassification quality summary

We now summarize the Phase 3 reclassifications to see **how many VUS/Pathogenic variants** were
flagged as **Benign (BA1)** or **Likely benign (BS1)**.

This will produce a **reclassification summary table**.


In [11]:
# Expect columns: ClinicalSignificance (original) + ProposedReclassification (from Phase 3)
if "ProposedReclassification" not in cands.columns:
    print("⚠️ ProposedReclassification not found in candidates; cannot build reclassification summary.")
else:
    reclass_summary = (
        cands.groupby(["ClinicalSignificance", "ProposedReclassification"])["variant_key"]
        .count()
        .reset_index()
        .rename(columns={"variant_key": "count"})
    )

    table4_reclass_path = os.path.join(RESULTS_TABLES, "table4_reclassification_summary.csv")
    reclass_summary.to_csv(table4_reclass_path, index=False)
    print("✅ Saved reclassification summary to:", table4_reclass_path)

    reclass_summary.head()


✅ Saved reclassification summary to: /content/drive/MyDrive/BRCA2-database-bias/results/tables/table4_reclassification_summary.csv


## 7. Summary

In this Phase 4 notebook you:

1. Quantified **population presence bias** (Table 2).  
2. Assessed **review status vs SAS AF** (Table 3 + Figure 4).  
3. Measured cross-population skew via **SAS/EUR ratio** (ratio summary + Figure 5).  
4. Summarized **ACMG-based reclassifications** from Phase 3 (reclassification summary).  

These outputs support your argument that:

- South Asian BRCA2 variants are underrepresented in standard databases.
- Some variants classified as VUS/Pathogenic are **too common in South Asians** to be plausibly high-penetrance.
